In [ ]:
import cv2
from doclayout_yolo import YOLOv10
path = '/Users/a2/Desktop/Bildschirmfoto 2024-11-01 um 00.46.51.png'
model = YOLOv10("/Users/a2/.cache/huggingface/hub/models--juliozhao--DocLayout-YOLO-DocStructBench/snapshots/8c3299a30b8ff29a1503c4431b035b93220f7b11/doclayout_yolo_docstructbench_imgsz1024.pt")

/Users/a2/anaconda3/envs/trade/lib/python3.12/site-packages/doclayout_yolo/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_locatio

In [ ]:
from doclayout_yolo.engine.results import Results
def generate(image, model:YOLOv10):
        
    # Load the pre-trained model


    det_res = model.predict(
        image,   # Image to predict
        imgsz=1024,        # Prediction image size
        conf=0.2,          # Confidence threshold
        device="cpu",    # Device to use (e.g., 'cuda:0' or 'cpu')
        verbose=False
    )   
    return det_res

{0: 'title', 1: 'plain text', 2: 'abandon', 3: 'figure', 4: 'figure_caption', 5: 'table', 6: 'table_caption', 7: 'table_footnote', 8: 'isolate_formula', 9: 'formula_caption'}

In [ ]:
### glaube hier wird immer noch die nächste caption gesucht obwohl diese näher an einer anderen box ist.

def match_table_caption_to_table(inputs):
    """
    Matches table captions to their corresponding tables based on proximity.

    Parameters:
    inputs (list): A list of dictionaries containing detected elements with their bounding boxes.

    Returns:
    list: A list of dictionaries with 'caption' and 'table' keys, representing matched pairs.
    """
    tables = [item for item in inputs if item['name'] == 'table']
    captions = [item for item in inputs if item['name'] == 'table_caption']

    matches = []

    for caption in captions:
        caption_box = caption['box']
        min_distance = float('inf')
        matched_table = None

        for table in tables:
            table_box = table['box']

            # Compute vertical distance between caption and table
            if caption_box['y2'] <= table_box['y1']:
                distance = table_box['y1'] - caption_box['y2']  # Caption is above the table
            elif caption_box['y1'] >= table_box['y2']:
                distance = caption_box['y1'] - table_box['y2']  # Caption is below the table
            else:
                distance = 0  # Overlapping vertically

            # Compute horizontal overlap
            x_overlap = max(0, min(caption_box['x2'], table_box['x2']) - max(caption_box['x1'], table_box['x1']))

            # Only consider captions that horizontally overlap with the table
            if x_overlap > 0 and distance < min_distance:
                min_distance = distance
                matched_table = table

        if matched_table:
            matches.append({'caption': caption, 'table': matched_table})

    return matches

In [ ]:
from PIL import Image
def extract_sub_image(path:str, coordinates)->None:
    image = Image.open(path)
    return image.crop(coordinates)
class TableImage:
    def __init__(self, path, x1, y1, x2, y2, caption="") -> None:
        self.coordinates = (int(x1), int(y1), int(x2+1), int(y2+1))
        self.img:Image = extract_sub_image(path, self.coordinates)
        self.caption=caption

In [60]:
import json
res = json.loads(generate(path, model)[0].tojson())
tables = [x for x in res if x["name"]=="table"]


In [61]:
box = tables[0]["box"]
TableImage(path, box["x1"], box["y1"], box["x2"], box["y2"]).img.show()


In [51]:
tables[0]

{'name': 'table',
 'class': 5,
 'confidence': 0.97744,
 'box': {'x1': 69.12816, 'y1': 756.10773, 'x2': 737.65228, 'y2': 1068.82764}}

In [14]:
type(res)

doclayout_yolo.engine.results.Results

In [62]:

res

[{'name': 'table',
  'class': 5,
  'confidence': 0.97744,
  'box': {'x1': 69.12816, 'y1': 756.10773, 'x2': 737.65228, 'y2': 1068.82764}},
 {'name': 'table',
  'class': 5,
  'confidence': 0.97463,
  'box': {'x1': 69.0334, 'y1': 277.42413, 'x2': 822.0285, 'y2': 503.14926}},
 {'name': 'plain text',
  'class': 1,
  'confidence': 0.96961,
  'box': {'x1': 65.7185, 'y1': 147.82533, 'x2': 979.5835, 'y2': 238.79291}},
 {'name': 'plain text',
  'class': 1,
  'confidence': 0.96861,
  'box': {'x1': 66.33956,
   'y1': 1172.43652,
   'x2': 977.52289,
   'y2': 1230.44641}},
 {'name': 'title',
  'class': 0,
  'confidence': 0.93105,
  'box': {'x1': 65.85232, 'y1': 563.1853, 'x2': 234.17126, 'y2': 586.26813}},
 {'name': 'title',
  'class': 0,
  'confidence': 0.92302,
  'box': {'x1': 66.71995, 'y1': 115.30495, 'x2': 151.06572, 'y2': 138.37685}},
 {'name': 'plain text',
  'class': 1,
  'confidence': 0.87674,
  'box': {'x1': 66.37093, 'y1': 593.40167, 'x2': 964.9295, 'y2': 685.55145}},
 {'name': 'table_cap

In [1]:
import pypdf

pp = "/Users/a2/code/fin/trade/test_docs/SELECTED TEXTILE IND. ASSOC. S.A. 01 - 11 - 2024.pdf"
doc =  pypdf.PdfReader(pp)

In [6]:
print(doc.get_page(0).extract_text())

Form Κ3-02-025
DATE: 01/11/2024 TIME: 12:00:00 πμ
α/α
Ελληνικό 
Σύμβολο / Greek 
Symbol
Αγγλικό 
Σύμβολο / 
English Symbol
Κωδικός ISIN / 
ISIN Code
Ενέργεια επί του Χρεογράφου / 
Action
Ωρα / 
Time
1 ΕΠΙΛΚ EPIL GRS045003001 Προσωρινή Αναστολή / Halt 12:00:00 πμ
Select one
Select one
Select one
Select one
Select one
Μετοχή/ 
Share: Επικεφαλίδα / Title:
Κείμενο / Text:
Από 01/11/2024, κατόπιν αιτήματος της Επιτροπής Κεφαλαιαγοράς, αναστέλλεται προσωρινά η διαπραγμάτευση των μετοχών της εταιρίας «ΕΠΙΛΕΚΤΟΣ 
ΚΛΩΣ/ΡΓΙΑ Α.Ε.Β.Ε.» (GRS045003001) στο Χρηματιστήριο Αθηνών. 
As of November 1, 2024, after consideration of the relevant request of the Hellenic Capital Market Commission, the trading of the shares of the company  
“SELECTED TEXTILE IND. ASSOC. S.A.” (GRS045003001) is temporarily suspended on ATHEX.
Select one
Select one
Select one
Περιγραφή / Description
ΕΠΙΛΚ / EPIL Αναστολή / Halt
Select one
ΔΙΑΚΟΠΗ / ΕΠΑΝΕΝΑΡΞΗ ΑΞΙΟΓΡΑΦΟΥ
HALT / RESUME SYMBOL
Διακοπή & Επαναφορά Αξιογράφου / Halt